In [1]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
TRAINING_DIR = r"C:\Users\laksh\TARP PROJECT\FIRE-SMOKE-DATASET\FIRE-SMOKE-DATASET\Train"
training_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.15,horizontal_flip=True,fill_mode='nearest')
VALIDATION_DIR = r"C:\Users\laksh\TARP PROJECT\FIRE-SMOKE-DATASET\FIRE-SMOKE-DATASET\Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(TRAINING_DIR,target_size=(224,224),shuffle = True,class_mode='categorical',batch_size = 128)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,target_size=(224,224),class_mode='categorical',shuffle = True,batch_size= 14)

Found 2700 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 20,validation_data = validation_generator,validation_steps = 14)

Epoch 1/20
14/14 [==============================] - 172s 11s/step - loss: 9.1838 - acc: 0.5555 - val_loss: 0.7642 - val_acc: 0.7857
Epoch 2/20
14/14 [==============================] - 368s 27s/step - loss: 0.5086 - acc: 0.8052 - val_loss: 0.7949 - val_acc: 0.6020
Epoch 3/20
14/14 [==============================] - 322s 22s/step - loss: 0.5569 - acc: 0.7868 - val_loss: 0.4981 - val_acc: 0.8265
Epoch 4/20
14/14 [==============================] - 378s 28s/step - loss: 0.4808 - acc: 0.8175 - val_loss: 0.3065 - val_acc: 0.8724
Epoch 5/20
14/14 [==============================] - 147s 10s/step - loss: 0.4645 - acc: 0.8305 - val_loss: 0.3128 - val_acc: 0.8980
Epoch 6/20
14/14 [==============================] - 154s 11s/step - loss: 0.4330 - acc: 0.8465 - val_loss: 0.2852 - val_acc: 0.8929
Epoch 7/20
14/14 [==============================] - 151s 11s/step - loss: 0.5632 - acc: 0.8150 - val_loss: 0.3104 - val_acc: 0.8929
Epoch 8/20
14/14 [==============================] - 161s 11s/step - loss: 0.

In [6]:
#To train the top 2 inception blocks, freeze the first 249 layers and unfreeze the rest.
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True
#Recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 10,validation_data = validation_generator,validation_steps = 14)

C:\Users\laksh\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
14/14 [==============================] - 157s 11s/step - loss: 0.6928 - acc: 0.6856 - val_loss: 0.2758 - val_acc: 0.9082
Epoch 2/10
14/14 [==============================] - 103s 7s/step - loss: 0.6428 - acc: 0.7132 - val_loss: 0.3017 - val_acc: 0.8929
Epoch 3/10
14/14 [==============================] - 97s 7s/step - loss: 0.5222 - acc: 0.7822 - val_loss: 0.2248 - val_acc: 0.9031
Epoch 4/10
14/14 [==============================] - 129s 9s/step - loss: 0.4690 - acc: 0.7971 - val_loss: 0.2908 - val_acc: 0.8878
Epoch 5/10
14/14 [==============================] - 117s 8s/step - loss: 0.3919 - acc: 0.8490 - val_loss: 0.3074 - val_acc: 0.8724
Epoch 6/10
14/14 [==============================] - 118s 8s/step - loss: 0.3806 - acc: 0.8467 - val_loss: 0.3064 - val_acc: 0.8827
Epoch 7/10
14/14 [==============================] - 108s 8s/step - loss: 0.3231 - acc: 0.8741 - val_loss: 0.3213 - val_acc: 0.8776
Epoch 8/10
14/14 [==============================] - 114s 8s/step - loss: 0.3149 - a

In [16]:
model.summary()
model.save(r'C:\Users\laksh\TARP PROJECT\InceptionV3.h5')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 111, 111, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                           

In [ ]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
#Load the saved model
model = tf.keras.models.load_model(r'C:\Users\laksh\TARP PROJECT\InceptionV3.h5')
video = cv2.VideoCapture(0)
while True:
        _, frame = video.read()
#Convert the captured frame into RGB
        im = Image.fromarray(frame, 'RGB')
#Resizing into 224x224 because we trained the model with this image size.
        im = im.resize((224,224))
        img_array = tf.keras.utils.img_to_array(im)
        img_array = np.expand_dims(img_array, axis=0) / 255
        probabilities = model.predict(img_array)[0]
        #Calling the predict method on model to predict 'fire' on the image
        prediction = np.argmax(probabilities)
        #if prediction is 0, which means there is fire in the frame.
        if prediction == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                print(probabilities[prediction])
cv2.imshow("Capturing", frame)
key=cv2.waitKey(1)
if key == ord('q'):
    break
video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 115ms/step


In [12]:
!pip install opencv-python -i http://pypi.douban.com/simple/ --trusted-host pypi.douban.com

Looking in indexes: http://pypi.douban.com/simple/


In [11]:
!pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
